In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
def make_hparam_string(learning_rate, conv1_depth,conv2_depth,fc1_size,fc2_size):

  return "lr_"+str(learning_rate)+",conv1depth_"+str(conv1_depth)+",conv2depth_"+str(conv2_depth)+",fc1size_"+str(fc1_size)+",fc2size_"+str(fc2_size)

In [11]:
make_hparam_string(1,2,3,4,5)

'lr_1,conv1depth_2,conv2depth_3,fc1size_4,fc2size_5'

In [3]:
def conv_layer(input,channels_in,channels_out,name ="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out] ,stddev=0.1),name ="W")
        b = tf.Variable(tf.constant(0.1,shape= [channels_out]),name ="B")
        conv = tf.nn.conv2d(input,w,strides = [1,1,1,1],padding="SAME")
        act = tf.nn.relu(conv+b)
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)

        return act

In [4]:
def fc_layer(input,channels_in,channels_out,name ="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([channels_in,channels_out],stddev=0.1),name ="W")
        b = tf.Variable(tf.constant(0.1,shape=[channels_out]),name="B")
        act = tf.nn.relu(tf.matmul(input,w) + b)

        return act

In [28]:
def train_mnist(learning_rate, conv1_depth,conv2_depth,fc1_size,fc2_size):
    tf.reset_default_graph()
    param_string  = make_hparam_string(learning_rate,conv1_depth,conv2_depth,fc1_size,fc2_size)
    print("Starting training for:"+param_string)
    
    x = tf.placeholder(tf.float32,shape=[None,784],name="x")
    y = tf.placeholder(tf.float32,shape=[None,10],name="labels")
    x_image = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',x_image,3)

    conv1 = conv_layer(x_image,1,conv1_depth,"conv1")
    pool1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

    conv2 = conv_layer(pool1,conv1_depth,conv2_depth,"conv2")
    pool2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    flattened = tf.reshape(pool2,[-1,7*7*conv2_depth])

    fc1 = fc_layer(flattened,7*7*conv2_depth,fc1_size,"fc1")
    logits = fc_layer(fc1,fc1_size,fc2_size,"fc2")

    with tf.name_scope("cross_entropy"):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
        tf.summary.scalar('cross_entropy',cross_entropy)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar("accuracy",accuracy)

    with tf.Session() as sess:
        merged_summary =  tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_demo/"+param_string)
        writer.add_graph(sess.graph)
        sess.run(tf.global_variables_initializer())

        for i in range(2001):
            batch = mnist.train.next_batch(100)

            if i % 500 == 0:
                [train_accuracy] = sess.run([accuracy],feed_dict =  {x:batch[0] , y:batch[1]})
                summary = sess.run(merged_summary , feed_dict={x:batch[0],y:batch[1]})
                print("step %d ,training accuracy %g" % (i,train_accuracy))
                writer.add_summary(summary,i)

            sess.run(train_step,feed_dict = {x:batch[0],y:batch[1]})

In [30]:
learning_rates = [1E-3,1E-4,1E-5]
conv1_depths = [16,32,64]
conv2_depths = [32,64,128]
fc1_sizes = [512,1024,2048]

for learning_rate in learning_rates:
    for conv1_depth in conv1_depths:
        for conv2_depth in conv2_depths:
            for fc1_size in fc1_sizes:
                train_mnist(learning_rate,conv1_depth,conv2_depth,fc1_size,10)

Starting training for:lr_0.001,conv1depth_16,conv2depth_32,fc1size_512,fc2size_10
step 0 ,training accuracy 0.14
step 500 ,training accuracy 0.22
step 1000 ,training accuracy 0.25
step 1500 ,training accuracy 0.37
step 2000 ,training accuracy 0.52
Starting training for:lr_0.001,conv1depth_16,conv2depth_32,fc1size_1024,fc2size_10
step 0 ,training accuracy 0.12
step 500 ,training accuracy 0.55
step 1000 ,training accuracy 0.59
step 1500 ,training accuracy 0.74
step 2000 ,training accuracy 1
Starting training for:lr_0.001,conv1depth_16,conv2depth_32,fc1size_2048,fc2size_10
step 0 ,training accuracy 0.03
step 500 ,training accuracy 0.05
step 1000 ,training accuracy 0.12
step 1500 ,training accuracy 0.08
step 2000 ,training accuracy 0.08
Starting training for:lr_0.001,conv1depth_16,conv2depth_64,fc1size_512,fc2size_10
step 0 ,training accuracy 0.03
step 500 ,training accuracy 0.09
step 1000 ,training accuracy 0.09
step 1500 ,training accuracy 0.07
step 2000 ,training accuracy 0.09
Starting 

step 0 ,training accuracy 0.1
step 500 ,training accuracy 0.38
step 1000 ,training accuracy 0.47
step 1500 ,training accuracy 0.53
step 2000 ,training accuracy 0.57
Starting training for:lr_0.0001,conv1depth_16,conv2depth_128,fc1size_1024,fc2size_10
step 0 ,training accuracy 0.1
step 500 ,training accuracy 0.18
step 1000 ,training accuracy 0.19
step 1500 ,training accuracy 0.13
step 2000 ,training accuracy 0.21
Starting training for:lr_0.0001,conv1depth_16,conv2depth_128,fc1size_2048,fc2size_10
step 0 ,training accuracy 0.13
step 500 ,training accuracy 0.12
step 1000 ,training accuracy 0.1
step 1500 ,training accuracy 0.12
step 2000 ,training accuracy 0.13
Starting training for:lr_0.0001,conv1depth_32,conv2depth_32,fc1size_512,fc2size_10
step 0 ,training accuracy 0.12
step 500 ,training accuracy 0.77
step 1000 ,training accuracy 0.77
step 1500 ,training accuracy 0.77
step 2000 ,training accuracy 0.71
Starting training for:lr_0.0001,conv1depth_32,conv2depth_32,fc1size_1024,fc2size_10
st

step 0 ,training accuracy 0.07
step 500 ,training accuracy 0.67
step 1000 ,training accuracy 0.89
step 1500 ,training accuracy 0.96
step 2000 ,training accuracy 0.93
Starting training for:lr_1e-05,conv1depth_32,conv2depth_64,fc1size_1024,fc2size_10
step 0 ,training accuracy 0.06
step 500 ,training accuracy 0.32
step 1000 ,training accuracy 0.44
step 1500 ,training accuracy 0.42
step 2000 ,training accuracy 0.57
Starting training for:lr_1e-05,conv1depth_32,conv2depth_64,fc1size_2048,fc2size_10
step 0 ,training accuracy 0.14
step 500 ,training accuracy 0.62
step 1000 ,training accuracy 0.67
step 1500 ,training accuracy 0.67
step 2000 ,training accuracy 0.75
Starting training for:lr_1e-05,conv1depth_32,conv2depth_128,fc1size_512,fc2size_10
step 0 ,training accuracy 0.06
step 500 ,training accuracy 0.49
step 1000 ,training accuracy 0.56
step 1500 ,training accuracy 0.78
step 2000 ,training accuracy 0.75
Starting training for:lr_1e-05,conv1depth_32,conv2depth_128,fc1size_1024,fc2size_10
ste

Launch tensorboard passing --logdir <path used in FileWriter> :
tensorboard --logdir /tmp/mnist_demo/2